In [1]:
#########################################
##The model deals with the prediction  ##
##predict required commodities for     ##
##each county each month and the total ##
## required for the next years         ##
#########################################

In [2]:
import pandas as pd

df  = pd.read_csv('data.csv')

df.head()

,periodid,periodname,periodcode,perioddescription,dataid,dataname,datacode,datadescription,Baringo County,Bomet County,...,Siaya County,Taita Taveta County,Tana River County,Tharaka Nithi County,Trans Nzoia County,Turkana County,Uasin Gishu County,Vihiga County,Wajir County,West Pokot County
0,202201,January 2022,202201,NaN,cPlWFYbBacW.w77uMi1KzOH,MOH 743 Rev2020_Rapid Diagnostic Tests Total Q...,NaN,NaN,2644.0,626.0,...,67556.0,496.0,4000.0,1148.0,4598.0,31277.0,3127.0,29950.0,1146.0,8189.0
1,202202,February 2022,202202,NaN,cPlWFYbBacW.w77uMi1KzOH,MOH 743 Rev2020_Rapid Diagnostic Tests Total Q...,NaN,NaN,3529.0,444.0,...,52890.0,983.0,2889.0,1398.0,5390.0,23382.0,2006.0,20458.0,1197.0,6908.0
2,202203,March 2022,202203,NaN,cPlWFYbBacW.w77uMi1KzOH,MOH 743 Rev2020_Rapid Diagnostic Tests Total Q...,NaN,NaN,2998.0,460.0,...,50940.0,1015.0,2392.0,671.0,6367.0,19729.0,2093.0,19161.0,885.0,6343.0
3,202204,April 2022,202204,NaN,cPlWFYbBacW.w77uMi1KzOH,MOH 743 Rev2020_Rapid Diagnostic Tests Total Q...,NaN,NaN,2755.0,508.0,...,41512.0,698.0,1567.0,509.0,5187.0,22980.0,1221.0,18753.0,936.0,9657.0
4,202205,May 2022,202205,NaN,cPlWFYbBacW.w77uMi1KzOH,MOH 743 Rev2020_Rapid Diagnostic Tests Total Q...,NaN,NaN,2878.0,355.0,...,63304.0,748.0,2239.0,1507.0,11820.0,35963.0,1595.0,34762.0,1236.0,13388.0


In [3]:
#create a timestamp(the format is YYYY-MM-DD)
datetime  = []

def get_datetime():
    datetime.clear()
    for timestamp in df['periodid']:

        dates = str(timestamp)[:4]+"-"+str(timestamp)[-2:]

        datetime.append(dates)

get_datetime()
#print(datetime)

['2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']


In [4]:
#per county and loop for all
#rename the columns
new_names  = {"Baringo County": 'baringo', "Bomet County": 'bomet', "Bungoma County": 'bungoma', "Busia County": 'busia', "Elgeyo Marakwet County": 'elgeyo',
               "Embu County": 'embu', "Garissa County": 'garissa', "Homa Bay County": 'homabay', "Isiolo County": 'isiolo', "Kajiado County": 'kajiado',
                "Kakamega County": 'kakamega', "Kericho County": 'kericho', "Kiambu County": 'kiambu', "Kilifi County": 'kilifi', "Kirinyaga County": 'kirinyaga',
                "Kisii County": 'kisii', "Kisumu County": 'kisumu', "Kitui County": 'kitui', "Kwale County": 'kwale', "Laikipia County": 'laikipia', "Lamu County": 'lamu',
                "Machakos County": 'machakos', "Makueni County": 'makueni', "Mandera County": 'mandera', "Marsabit County": 'marsabit', "Meru County": 'meru', "Migori County": 'migori',
                "Mombasa County": 'mombasa', "Muranga County": 'muranga', "Nairobi County": 'nairobi', "Nakuru County": 'nakuru', "Nandi County": 'nandi', "Narok County": 'narok',
                "Nyamira County": 'nyamira', "Nyandarua County": 'nyandarua', "Nyeri County": 'nyeri', "Samburu County": 'samburu', "Siaya County": 'siaya', "Taita Taveta County": 'taita', 
                "Tana River County": 'tanariver', "Tharaka Nithi County": 'tharaka', "Trans Nzoia County": 'transzoia', "Turkana County": 'turkana', "Uasin Gishu County": 'uasingishu',
                "Vihiga County": 'vihiga', "Wajir County": 'wajir', "West Pokot County": 'westpokot'}

df.rename(columns=new_names, inplace=True)

years = [ periodid for periodid in df['periodid']]

In [39]:
#create a data to be supported during prediction(the format is y)
from math import floor
county_monthly_data = []

def county_year_consumption(county):
   
    """This county_year_consumption function gives out the total commodities dispatched a certain complete year
       for a particular county
    """
    #run above function to get the year periods in place
    get_datetime()
    county_monthly_data.clear()
    try:
      for periodid in years:
         val = df.loc[df['periodid']==int(periodid)].sum()
         
         commodity_total = val[county]
         if commodity_total == 0:
            county_monthly_data.append(0)
         else:
            county_monthly_data.append(floor(commodity_total))
         #print(f"{county} has {commodity_total} kits in {periodid}")
      
       
    except Exception as e:
       
       if e.__class__.__name__ == 'KeyError':
          print("Check if the county name is accurate.Or ask the lead developer for solution.")

county_year_consumption('kirinyaga')



[112, 126, 175, 66, 74, 26, 24, 29, 69, 69, 68, 127, 19, 456, 138, 157, 242, 203, 172, 145, 116, 117, 212, 108, 97, 78, 77, 72, 57, 31, 112, 24, 20, 0, 0, 3, 36, 9, 54, 17, 40, 0, 3, 14, 163, 98, 56, 54, 210, 164, 219, 254, 152, 107, 164, 78, 179, 60, 19, 54]


In [40]:
import numpy as np
#create a dataframe
#Have the data in a dictionary 
#the first key will be ds while the other y
# Create a dataframe with a datetime column and a corresponding numeric column

#convert the list of datetime to a pandas datetime object
datetime_index  = pd.to_datetime(datetime, format='%Y-%m')
#Extract the end month of the month for each month
end_of_month = datetime_index + pd.offsets.MonthBegin(1)
#print(end_of_month)
dataframe  = {'ds':end_of_month , 'y': county_monthly_data}

new_df = pd.DataFrame(dataframe)

#new_df.head()


In [41]:
#predict using the prophet
from prophet import Prophet
from prophet.diagnostics import performance_metrics, cross_validation

# Instantiate a Prophet object and fit the model to the data
model  = Prophet()

model.fit(new_df)

# Create a dataframe with dates to make a prediction for
future  = model.make_future_dataframe(periods=6, freq='MS')

# Use the model to make a prediction for the next 12 months
forecast  = model.predict(future)

#print the predicted values for the next 12 months
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7))
#model.plot(forecast)

"""#calculate performance metrics using cross-validation
df_cv  = cross_validation(model, initial='730 days', period='180 days', horizon='365 days')

df_p = performance_metrics(df_cv)

#print the performance metrics

print(df_p)"""

12:56:35 - cmdstanpy - INFO - Chain [1] start processing
12:56:36 - cmdstanpy - INFO - Chain [1] done processing


           ds        yhat  yhat_lower  yhat_upper
59 2023-01-01   74.359392  -20.484325  166.057535
60 2023-02-01  102.134585   14.330211  197.219903
61 2023-03-01  116.931968   25.235567  214.340637
62 2023-04-01  173.707020   79.671587  272.499378
63 2023-05-01  137.026558   35.743772  224.878335
64 2023-06-01  126.431857   33.134592  223.053869
65 2023-07-01   71.095873  -17.739074  165.425560


"#calculate performance metrics using cross-validation\ndf_cv  = cross_validation(model, initial='730 days', period='180 days', horizon='365 days')\n\ndf_p = performance_metrics(df_cv)\n\n#print the performance metrics\n\nprint(df_p)"